In [1]:
import sklearn
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import seaborn as sns

In [2]:
!pip install trimap
!pip install pymde
!pip install umap-learn[plot]


import getpass
import os
from os import path

import trimap
import pymde
import umap

import random
import numpy as np
import matplotlib.pyplot as plt
from time import time
from matplotlib.ticker import NullFormatter
import pandas as pd

from sklearn import manifold, datasets
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

from umap import UMAP

import tensorflow as tf

from tensorflow import keras

from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

def apply_panel_of_manifold_learning_methods(X,color,
                                Color_by_branches=[],precomputed_results={},
                                color_map='cool',ColorByFeature='',
                                variable_names=[],ElMapFolder='',
                                n_neighbors=20, n_components = 2,
                                title_fontsize = 30,points_size = 30,
                                methods_to_apply=[],
                                n_subplots_x = 4, n_subplots_y = 3,
                                figsizex = 20, figsizey =20):
    viz_results = precomputed_results
    #Set figure parameters
    n_points = X.shape[0]
    
    #cmap = plt.cm.Paired
    #cmap = 'hot'
    cmap = color_map
    # cmap = plt.cm.tab20
    plt.style.use('ggplot')
    fig = plt.figure(figsize=(figsizex, figsizey))
    applyAllMethods = True
    if len(methods_to_apply)>0:
        applyAllMethods = False

    color_seq = [[1,0,0],[0,1,0],[0,0,1],[0,1,1],[1,0,1],[1,1,0],
             [1,0,0.5],[1,0.5,0],[0.5,0,1],[0.5,1,0],
             [0.5,0.5,1],[0.5,1,0.5],[1,0.5,0.5],
             [0,0.5,0.5],[0.5,0,0.5],[0.5,0.5,0],[0.5,0.5,0.5],[0,0,0.5],[0,0.5,0],[0.5,0,0],
             [0,0.25,0.5],[0,0.5,0.25],[0.25,0,0.5],[0.25,0.5,0],[0.5,0,0.25],[0.5,0.25,0],
             [0.25,0.25,0.5],[0.25,0.5,0.25],[0.5,0.25,0.25],[0.25,0.25,0.5],[0.25,0.5,0.25],
             [0.25,0.25,0.5],[0.25,0.5,0.25],[0.5,0,0.25],[0.5,0.25,0.25]]*10
    color_seq = color_seq[:len(set(color))]

    color1 = color
    if len(Color_by_branches)>0:
        #color1 = vec_labels_by_branches
        color2 = Color_by_branches
        color2_unique, color2_count = np.unique(color2, return_counts=True)
        inds = sorted(range(len(color2_count)), key=lambda k: color2_count[k], reverse=True)
        newc = []
        for i,c in enumerate(color2):
            k = np.where(color2_unique==c)[0][0]
            count = color2_count[k]
            k1 = np.where(inds==k)[0][0]
            k1 = k1%len(color_seq)
            col = color_seq[k1]
            newc.append(col)
        color2 = newc
        color1 = color2

    if not ColorByFeature=='':
        k = variable_names.index(ColorByFeature)
        #color1 = X_original[:,k]
        color1 = X[:,k]

    onlyDraw = not len(precomputed_results)==0

    print('Start computations...')

    # some standard methods
    i = 1

    ####################### PCA #########################
    if applyAllMethods or 'PCA' in methods_to_apply:
        pca = PCA(n_components=n_components)
        t0 = time()
        if  not onlyDraw or not 'PCA' in precomputed_results:
            Y_PCA = pca.fit_transform(X)
            viz_results['PCA'] = Y_PCA
        else:
            Y_PCA = precomputed_results['PCA']
        t1 = time()
        print("PCA: %.2g sec" % (t1 - t0))

        ax = fig.add_subplot(n_subplots_x,n_subplots_y,i)
        sns.scatterplot(x = Y_PCA[:, 0], y = Y_PCA[:, 1], hue = color1, palette=color_seq, legend = 'full')
        plt.title("PCA",fontdict = {'fontsize' : title_fontsize})
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        plt.axis('tight')


    ### LLE ###
    if applyAllMethods or 'LLE' in methods_to_apply:
        t0 = time()
        if  not onlyDraw or not 'LLE' in precomputed_results:
            print('Computing LLE...')
            Y_LLE = manifold.LocallyLinearEmbedding(n_neighbors=n_neighbors, n_components=n_components,
                                    eigen_solver='auto',
                                    method='standard').fit_transform(X)
            viz_results['LLE'] = Y_LLE
        else:
             Y_LLE = viz_results['LLE']
        t1 = time()
        print("%s: %.2g sec" % ('LLE', t1 - t0))
        i+=1
        ax = fig.add_subplot(n_subplots_x,n_subplots_y,i)
        sns.scatterplot(x = Y_LLE[:, 0], y = Y_LLE[:, 1], hue = color1, palette=color_seq, legend = 'full')
        plt.title("LLE",fontdict = {'fontsize' : title_fontsize})
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        plt.axis('tight')


    ### Modified LLE ###
    if applyAllMethods or 'MLLE' in methods_to_apply:
        t0 = time()
        if  not onlyDraw  or not 'MLLE' in precomputed_results:
            Y_MLLE = manifold.LocallyLinearEmbedding(n_neighbors=n_neighbors, n_components=n_components,
                                        eigen_solver='auto',
                                        method='modified').fit_transform(X)
            viz_results['MLLE'] = Y_MLLE
        else:
            Y_MLLE = viz_results['MLLE']
        t1 = time()
        print("%s: %.2g sec" % ('Modified LLE', t1 - t0))
        i+=1
        ax = fig.add_subplot(n_subplots_x,n_subplots_y,i)
        sns.scatterplot(x = Y_MLLE[:, 0], y = Y_MLLE[:, 1], hue = color1, palette=color_seq, legend = 'full')
        plt.title("MLLE",fontdict = {'fontsize' : title_fontsize})
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        plt.axis('tight')


    ### ISOMAP ###
    if applyAllMethods or 'ISOMAP' in methods_to_apply:
        i += 1
        t0 = time()
        if  not onlyDraw or not 'ISOMAP' in precomputed_results:
            Y_ISOMAP = manifold.Isomap(n_neighbors=n_neighbors, n_components=n_components).fit_transform(X)
            viz_results['ISOMAP'] = Y_ISOMAP
        else:
            Y_ISOMAP = viz_results['ISOMAP']
        t1 = time()
        print("Isomap: %.2g sec" % (t1 - t0))
        ax = fig.add_subplot(n_subplots_x,n_subplots_y,i)
        sns.scatterplot(x = Y_ISOMAP[:, 0], y = Y_ISOMAP[:, 1], hue = color1, palette=color_seq, legend = 'full')        
        plt.title("Isomap",fontdict = {'fontsize' : title_fontsize})
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        plt.axis('tight')


    ### MDS ###
    if applyAllMethods or 'MDS' in methods_to_apply:    
        i += 1
        t0 = time()
        if  not onlyDraw or not 'MDS' in precomputed_results:
            mds = manifold.MDS(n_components, max_iter=100, n_init=1)
            Y_MDS = mds.fit_transform(X)
            viz_results['MDS'] = Y_MDS
        else:
            Y_MDS = viz_results['MDS']
        t1 = time()
        print("MDS: %.2g sec" % (t1 - t0))
        ax = fig.add_subplot(n_subplots_x,n_subplots_y,i)
        sns.scatterplot(x = Y_MDS[:, 0], y = Y_MDS[:, 1], hue = color1, palette=color_seq, legend = 'full')                
        plt.title("MDS",fontdict = {'fontsize' : title_fontsize})
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        plt.axis('tight')
        
        
    ### SpectralEmbedding ###
    if applyAllMethods or 'SE' in methods_to_apply:      
        i += 1
        t0 = time()
        if  not onlyDraw  or not 'SE' in precomputed_results:
            se = manifold.SpectralEmbedding(n_components=n_components,n_neighbors=n_neighbors)
            Y_se = se.fit_transform(X)
            viz_results['SE'] = Y_se
        else:
            Y_se = viz_results['SE']
        t1 = time()
        print("SpectralEmbedding: %.2g sec" % (t1 - t0))
        ax = fig.add_subplot(n_subplots_x,n_subplots_y,i)
        sns.scatterplot(x = Y_se[:, 0], y = Y_se[:, 1], hue = color1, palette=color_seq, legend = 'full')                        
        plt.title("SpectralEmbedding",fontdict = {'fontsize' : title_fontsize})
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        plt.axis('tight')



    ### t-SNE ###
    if applyAllMethods or 'TSNE' in methods_to_apply:      
        i += 1
        t0 = time()
        if  not onlyDraw  or not 'TSNE' in precomputed_results:
            tsne = manifold.TSNE(n_components=n_components, random_state=0)
            Y_TSNE = tsne.fit_transform(X)
            viz_results['TSNE'] = Y_TSNE
        else:
            Y_TSNE = viz_results['TSNE']
        t1 = time()
        print("t-SNE: %.2g sec" % (t1 - t0))
        ax = fig.add_subplot(n_subplots_x,n_subplots_y,i)
        sns.scatterplot(x = Y_TSNE[:, 0], y = Y_TSNE[:, 1], hue = color1, palette=color_seq, legend = 'full')
        plt.title("t-SNE",fontdict = {'fontsize' : title_fontsize})
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        plt.axis('tight')


        
    ### UMAP ###
    if applyAllMethods or 'UMAP' in methods_to_apply:          
        i += 1
        t0 = time()
        if  not onlyDraw  or not 'UMAP' in precomputed_results:
            um = UMAP(n_neighbors=n_neighbors,
                  n_components=n_components)
            Y_UMAP = um.fit_transform(X)
            viz_results['UMAP'] = Y_UMAP
        else:
            Y_UMAP = viz_results['UMAP']
        t1 = time()
        print("UMAP: %.2g sec" % (t1 - t0))
        ax = fig.add_subplot(n_subplots_x,n_subplots_y,i)
        sns.scatterplot(x = Y_UMAP[:, 0], y = Y_UMAP[:, 1], hue = color1, palette=color_seq, legend = 'full')                                
        plt.title("UMAP",fontdict = {'fontsize' : title_fontsize})
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        plt.axis('tight')
        
    ### TRIMAP ###
    if applyAllMethods or 'TRIMAP' in methods_to_apply:              
        t0 = time()
        if  not onlyDraw  or not 'TRIMAP' in precomputed_results:
            Y_TRIMAP = trimap.TRIMAP(verbose=False).fit_transform(X)
            viz_results['TRIMAP'] = Y_TRIMAP
        else:
            Y_TRIMAP = viz_results['TRIMAP']
        t1 = time()
        print("TRIMAP: %.2g sec" % (t1 - t0))
        i += 1
        ax = fig.add_subplot(n_subplots_x,n_subplots_y,i)
        sns.scatterplot(x = Y_TRIMAP[:, 0], y = Y_TRIMAP[:, 1], hue = color1, palette=color_seq, legend = 'full')                                
        plt.title("TRIMAP",fontdict = {'fontsize' : title_fontsize})
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        plt.axis('tight')

        
    ### MDE ###
    if applyAllMethods or 'MDE' in methods_to_apply:              
        t0 = time()
        if  not onlyDraw  or not 'MDE' in precomputed_results:
            Y_MDE = pymde.preserve_neighbors(X, embedding_dim=2, verbose=False).embed()        
            viz_results['MDE'] = Y_MDE
        else:
            Y_MDE = viz_results['MDE']
        t1 = time()
        print("MDE: %.2g sec" % (t1 - t0))
        i += 1
        ax = fig.add_subplot(n_subplots_x,n_subplots_y,i)
        sns.scatterplot(x = Y_MDE[:, 0], y = Y_MDE[:, 1], hue = color1, palette=color_seq, legend = 'full')                                        
        plt.title("MDE",fontdict = {'fontsize' : title_fontsize})
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        plt.axis('tight')
                

    ### Autoencoder ###
    if applyAllMethods or 'AUTOENCODER' in methods_to_apply:        
        layer_sizes = [64,32,16,8]
        #encoder
        inputs = Input(shape=(X.shape[1],), name='encoder_input')
        x = inputs
        for size in layer_sizes:
            x = Dense(size, activation='relu',kernel_initializer='he_uniform')(x)
        latent = Dense(n_components,kernel_initializer='he_uniform', name='latent_vector')(x)
        encoder = Model(inputs, latent, name='encoder')

        #decoder
        latent_inputs = Input(shape=(n_components,), name='decoder_input')
        x = latent_inputs
        for size in layer_sizes[::-1]:
            x = Dense(size, activation='relu',kernel_initializer='he_uniform')(x)
        outputs = Dense(X.shape[1] ,activation='sigmoid',kernel_initializer='he_uniform',name='decoder_output')(x)
        decoder = Model(latent_inputs, outputs, name='decoder')

        #autoencoder
        autoencoder = Model(inputs, decoder(encoder(inputs)), name='autoencoder')

        #model summary
        # encoder.summary()
        # decoder.summary()
        # autoencoder.summary()
        X_01 = (X-X.min())/(X.max()-X.min())
        autoencoder.compile(loss='mse', optimizer='adam')
        t0 = time()
        if  not onlyDraw  or not 'AUTOENCODER' in precomputed_results:
            autoencoder.fit(x=X_01,y=X_01,epochs=200,verbose=0)
            Y_AUTOENCODER = encoder.predict(X_01)
            viz_results['AUTOENCODER'] = Y_AUTOENCODER
        else:
            Y_AUTOENCODER = viz_results['AUTOENCODER']
        t1 = time()
        print("Autoencoder: %.2g sec" % (t1 - t0))

        i += 1
        ax = fig.add_subplot(n_subplots_x,n_subplots_y,i)
        sns.scatterplot(x = Y_AUTOENCODER[:, 0], y = Y_AUTOENCODER[:, 1], hue = color1, palette=color_seq, legend = 'full')                                        
        plt.title("Autoencoder",fontdict = {'fontsize' : title_fontsize})
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        plt.axis('tight')        

    ### VAE ###
    if applyAllMethods or 'VAE' in methods_to_apply:        
        def sampling(args):
            z_mean, z_log_var = args
            epsilon = K.random_normal(shape=(n_components,))
            return z_mean + K.exp(z_log_var) * epsilon

        layer_sizes = [64,32,16,8]
        #encoder
        inputs = Input(shape=(X.shape[1],), name='encoder_input')
        x = inputs
        for size in layer_sizes:
            x = Dense(size, activation='relu',kernel_initializer='he_uniform')(x)

        z_mean = Dense(n_components,kernel_initializer='he_uniform', name='latent_mean')(x)
        z_log_var = Dense(n_components,kernel_initializer='he_uniform', name='latent_sigma')(x)

        z = Lambda(sampling, output_shape=(n_components,))([z_mean, z_log_var])
        encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

        #decoder
        latent_inputs = Input(shape=(n_components,), name='decoder_input_sampling')
        x = latent_inputs
        for size in layer_sizes[::-1]:
            x = Dense(size, activation='relu',kernel_initializer='he_uniform')(x)
        outputs = Dense(X.shape[1] ,activation='sigmoid',kernel_initializer='he_uniform',name='decoder_output')(x)
        decoder = Model(latent_inputs, outputs, name='decoder')

        #autoencoder
        vae = Model(inputs, decoder(encoder(inputs)[2]), name='vae')

        def vae_loss(x, x_decoded_mean):
            xent_loss = K.mean(K.square((x- x_decoded_mean)))
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
            print(type(xent_loss))
            print(type(kl_loss))
            return K.sum(xent_loss,kl_loss)
            #return tf.convert_to_tensor(kl_loss)
            #return xent_loss
        vae.compile(optimizer='adam', loss=vae_loss)

        X_01 = (X-X.min())/(X.max()-X.min())
        #print(X_01)
        X_01 = X.copy()
        t0 = time()
        if  not onlyDraw  or not 'VAE' in precomputed_results:
            vae.fit(x=X_01,y=X_01,epochs=200,verbose=0)
            Y_VAE = encoder.predict(X_01)[0]
            viz_results['VAE'] = Y_VAE
        else:
            Y_VAE = viz_results['VAE']
        t1 = time()
        print("VAE: %.2g sec" % (t1 - t0))
        i += 1
        ax = fig.add_subplot(n_subplots_x,n_subplots_y,i)
        sns.scatterplot(x = Y_VAE[:, 0], y = Y_VAE[:, 1], hue = color1, palette=color_seq, legend = 'full')                                                
        plt.title("VAE",fontdict = {'fontsize' : title_fontsize})
        ax.xaxis.set_major_formatter(NullFormatter())
        ax.yaxis.set_major_formatter(NullFormatter())
        plt.axis('tight')

    plt.tight_layout()
    
    return viz_results


     |████████████████████████████████| 5.6 MB 3.8 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 646 kB 48.7 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for trimap: filename=trimap-1.0.15-py3-none-any.whl size=14648 sha256=a4bdb84f54faa793703bdd5a7458760e614bb5a174e99ca304feb77b7697c96d
  Stored in directory: /home/mathildedacruz/.cache/pip/wheels/55/34/37/0d9dcba4884ea0b9730fa86378fb75ac32a16de411193baf41
  Created wheel for annoy: filename=annoy-1.17.0-cp38-cp38-linux_x86_64.whl size=577507 sha256=ac7132015395fef7562447e58efc841514f756eaf540d24d0d00aaae6ba43ae8
  Stored in directory: /home/mathildedacruz/.cache/pip/wheels/77/01/de/4421524f9997a25dfa7291121565d12ef514154945e80e907a
Successfully built trimap annoy
You should consider upgrading via the '/home/mathildedacruz/anaconda3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 135 kB 3.8 MB/s            
     |

     |████████████████████████████████| 15.8 MB 7.6 MB/s            
     |████████████████████████████████| 4.3 MB 35.8 MB/s            
     |████████████████████████████████| 1.6 MB 21.4 MB/s            
     |████████████████████████████████| 85 kB 13.3 MB/s            
     |████████████████████████████████| 76 kB 12.5 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 865 kB 30.7 MB/s            
     |████████████████████████████████| 40 kB 12.4 MB/s            
     |████████████████████████████████| 12.9 MB 14.7 MB/s            


  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82708 sha256=34df380239fef9bb2769d3b575df7f22cb2aeedf0d7041feec8819bb5dd50d18
  Stored in directory: /home/mathildedacruz/.cache/pip/wheels/f2/64/75/df601da9514261c8cb0830b9515d2b94b5a51f09ddeae92b9e
  Created wheel for datashape: filename=datashape-0.5.2-py3-none-any.whl size=59438 sha256=ad1e2e2687eaabb678d41ec8b3adf9d9f5634b90e9509957b5c8b2d739563932
  Stored in directory: /home/mathildedacruz/.cache/pip/wheels/6d/79/c4/c425774559165f472d32e5ef592ff9a71179abb31f05dbc98b
Successfully built umap-learn datashape
You should consider upgrading via the '/home/mathildedacruz/anaconda3/bin/python -m pip install --upgrade pip' command.


In [6]:
import random
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn import datasets
import umap
import zipfile
from urllib.request import urlopen
import io
import time
from mpl_toolkits.mplot3d import Axes3D


#dataset_name = 'lotto'
#dataset_name = 'Waterstress'
#dataset_name = 'cpu_small'
#dataset_name = 'jungle_chess_2pcs_endgame_elephant_elephant'
#dataset_name = 'Indian_pines'
#dataset_name = 'USPS'
#dataset_name = 'pol'
#dataset_name = 'mv'
#dataset_name = 'mnist_784'
file_location = 'https://raw.githubusercontent.com/auranic/ClinTrajan/master/data/infarction/infarctus_na_v30s20.txt'
df = pd.read_csv(io.BytesIO(urlopen(file_location).read()), header=0, sep='\t', quotechar='"', error_bad_lines=False)
X = df.to_numpy()[:,1:-2]
y = df.to_numpy()[:,-1]
#X, y = fetch_openml(dataset_name, version='1', return_X_y=True, as_frame=False)
print('Dataset size=',X.shape,type(X))

# downsample if needed
max_number_of_points = 10000
if X.shape[0]>max_number_of_points:
    randomRows = random.sample(range(X.shape[0]), max_number_of_points)
    X = X[randomRows,:]
    y = y[randomRows]

if 'str' in str(type(y[0])):
    unique_vals = list(np.unique(y))
    y = np.array([unique_vals.index(v) for v in y])


color = y
viz_results = {}

Dataset size= (1574, 114) <class 'numpy.ndarray'>


/tmp/ipykernel_15666/115950372.py:28: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(io.BytesIO(urlopen(file_location).read()), header=0, sep='\t', quotechar='"', error_bad_lines=False)


In [7]:
viz_results = apply_panel_of_manifold_learning_methods(X,color,
                                                       methods_to_apply=['PCA','UMAP','TRIMAP','MDE','TSNE',
                                                                        'LLE','MLLE','ISOMAP','MDS','SE', 
                                                                         'AUTOENCODER'])

Start computations...


TypeError: 'module' object is not callable

<Figure size 1440x1440 with 0 Axes>